# Introduction to SQL

### Overview of the SQL query language

The **Structured Query Language (SQL)** has several parts:

1. **Data-definition language (DDL)**; the SQL DDL provides commands for defining relation schemas (e.g. ``student (age, name, sex, ...)``), deleting relations, and modifying relation schemas. Specifically with commands for:
    * **Integrity**; specifying integrity constraints that the data stored in the database must satisfy. Updates that violate integrity constraints are disallowed.
    * **View definition**;
2. **Data-manipulation language (DML)**; the SQL DML provides the ability to query information from the database and to insert tuples into, delete tuples from, and modify tuples in a database. Here, it seems crucial to point out that SQL allows for:
    * **transaction control**; a transaction is a unit of work performed against a database, such as update or delete; SQL contains commands to control whether the transactions are actually concluded or not which is useful to guarantee the integrity of the database, namely via commands such as ``COMMIT`` or ``ROLL BACK``